<a href="https://colab.research.google.com/github/AbbyNyakara/Mushroom-Classification/blob/main/STT_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DEPENDACIES

In [11]:
# Import libraries:

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import os
import cv2

In [6]:
# No gpu connected?? yikes
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)

[]


In [7]:
# Setup GPU memory consumption growth: Avoid the OOM error:

# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#   tf.config.experimental.set_memory_growth(gpu, True)


# Load the data

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Load data

dataset = tf.keras.preprocessing.image_dataset_from_directory('/content/drive/MyDrive/Statistics/Project/Dataset')

Found 28759 files belonging to 2 classes.


## Split the data:

In [12]:
# Training set (80% of the data)
train_ds = tf.keras.utils.image_dataset_from_directory(
    '/content/drive/MyDrive/Statistics/Project/Dataset',
    validation_split=0.2,
    subset="training",
    seed=123
)

# Validation set(20% of the data)
validation_ds = tf.keras.utils.image_dataset_from_directory(
    '/content/drive/MyDrive/Statistics/Project/Dataset',
    validation_split=0.2,
    subset="validation",
    seed=123
)


Found 28759 files belonging to 2 classes.
Using 23008 files for training.
Found 28759 files belonging to 2 classes.
Using 5751 files for validation.
